## data_cleaning.ipynb

Performs data cleaning for restaurant reviews collected via web scraping. It includes extracting structured information, handling missing values, checking for duplicates, and preparing the data for further analysis.


In [2]:
import pandas as pd
import numpy as np
import re
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


### Select the raw data to process

In [3]:
raw_data_path = '../data/raw/'

name = '17punto10'
reviews_raw = pd.read_csv(raw_data_path + 'collected_reviews_' + name + '.csv')
resumme_raw = pd.read_csv(raw_data_path + 'resumme_' + name + '.csv')
display(resumme_raw)
display(reviews_raw.sample(5))

,stars,reviews
0,5,571
1,4,191
2,3,35
3,2,13
4,1,13


,author,local_guide_info,rating,review,date_text,text_backup
7,Cristina Q,9 reseñas · 14 fotos,5 estrellas,Sitio de nivel. La comida relación calidad-pre...,Hace 8 meses,Cristina Q\n9 reseñas · 14 fotos\n\n\n\n\n...
19,Andrea Cabello,Local Guide · 118 reseñas · 72 fotos,5 estrellas,"Espectacular comida, trato y ambiente. Servici...",Hace 11 meses,Andrea Cabello\nLocal Guide · 118 reseñas · 72...
1,Beatriz N.,Local Guide · 28 reseñas · 8 fotos,5 estrellas,"La comida es excepcional! Las migas, las croqu...",Hace 2 meses,Beatriz N.\nLocal Guide · 28 reseñas · 8 fotos...
21,Jonathan LG,Local Guide · 34 reseñas · 96 fotos,5 estrellas,Fuimos el fin de semana a Zaragoza y nos gusta...,Hace un año,Jonathan LG\nLocal Guide · 34 reseñas · 96 fot...
3,Carlos Casado,16 reseñas · 30 fotos,5 estrellas,Espacio confortable aún estando en la barra ja...,Hace 4 meses,Carlos Casado\n16 reseñas · 30 fotos\n\n\n\...


### Search words selected

Define a dictionary of regular expressions to extract specific fields (service, meal type, price range, scores, etc.) from the review text.

In [4]:
restaurant_search_words = {
    'service': r'Servicio\n([^\n]+)',
    'meal_type': r'Tipo de comida\n([^\n]+)',
    'price_per_person': r'Precio por persona\n([0-9€\- ]+)',
    'food_score': r'Comida: (\d+)',
    'service_score': r'Servicio: (\d+)',
    'atmosphere_score': r'Ambiente: (\d+)',
    'recommended': r'Platos recomendados\n([^\n]+)'
}

In [5]:
def extractRestaurantDetailsFromReview(sample, search_words=None, verbose=False):
    clean_text = re.sub(r'\\ue[0-9a-f]{3}', '', sample)
    clean_text = re.sub(r'\n+', '\n', clean_text)
    clean_text = clean_text.strip()

    # Store extracted values
    extracted_values = []

    # Loop through search words to extract values dynamically
    for key, regex in search_words.items():
        match = re.search(regex, clean_text)
        value = match.group(1) if match else ''
        extracted_values.append(value)

    return extracted_values

def applyExtractDetails(df, search_words=None):
    column_names = list(search_words.keys())
    df[column_names] = df['text_backup'].apply(lambda x: pd.Series(extractRestaurantDetailsFromReview(x, search_words=search_words)))
    return df

def extractReviewCount(text):
    if isinstance(text, str):  # Verify if its a string
        match = re.search(r'(\d+)\s+reseñas', text)
        if match:
            return int(match.group(1))
    return None

def extractStarRating(text):
    match = re.search(r'(\d+)\s+estrellas', text)
    if match:
        return int(match.group(1))
    return None

def extractRecommendations(recommendations):
    recommendations_list = recommendations.split(', ')
    if ' y ' in recommendations_list[-1]:
        last_dishes = recommendations_list[-1].rsplit(' y ', 1)
        recommendations_list = recommendations_list[:-1] + last_dishes
    return recommendations_list

def convertToDate(date_text):
    today = datetime.today()

    if 'semana' in date_text:
        # Extract number of weeks, default to 1 if no number is present
        weeks = pd.Series(date_text).str.extract(r'(\d+)')[0]
        weeks = int(weeks.iloc[0]) if pd.notna(weeks.iloc[0]) else 1
        monday_of_current_week = today - timedelta(days=today.weekday())  # Get Monday of the current week
        return monday_of_current_week.date() - timedelta(weeks=weeks)

    elif 'mes' in date_text:
        # Extract number of months, default to 1 if no number is present
        months = pd.Series(date_text).str.extract(r'(\d+)')[0]
        months = int(months.iloc[0]) if pd.notna(months.iloc[0]) else 1
        target_date = today - relativedelta(months=months)
        # Return the first day of the target month
        return target_date.replace(day=1).date()

    elif 'año' in date_text:
        # Extract number of years, default to 1 if no number is present
        years = pd.Series(date_text).str.extract(r'(\d+)')[0]
        years = int(years.iloc[0]) if pd.notna(years.iloc[0]) else 1
        target_date = today - relativedelta(years=years)
        # Return the first day of the target year
        return target_date.replace(month=1, day=1).date()

    return None  # Return None if no match is found


In [15]:
reviews = reviews_raw.copy()

### Removing duplicates

Check for duplicated rows in the dataset and remove them to ensure data integrity

In [16]:
# Convert any list-like columns to strings so they can be checked for duplicates
check_dups = reviews.copy()
for col in check_dups.columns:
    if check_dups[col].dtype == 'object' and isinstance(check_dups[col].iloc[0], list):
        check_dups[col] = check_dups[col].apply(lambda x: str(x))

# Now you can check and remove duplicates
duplicates_count = check_dups.duplicated().sum()
print(f"Number of duplicated rows: {duplicates_count}")

# Remove duplicates
reviews.drop_duplicates(inplace=True)
print("Duplicates removed successfully.")

Number of duplicated rows: 0
Duplicates removed successfully.


### Prepare and process all fields

Clean and convert relevant columns to numeric types, extract additional details (e.g., average price per person), and drop unnecessary columns from the DataFrame.

In [17]:
reviews['local_guide_reviews'] = reviews['local_guide_info'].apply(extractReviewCount)
reviews['rating_score'] = reviews['rating'].apply(extractStarRating)
reviews = applyExtractDetails(reviews, search_words = restaurant_search_words)
reviews['recommendations_list'] = reviews['recommended'].apply(extractRecommendations)
reviews['date'] = reviews['date_text'].apply(convertToDate)

reviews['food_score'] = reviews['food_score'].apply(pd.to_numeric, errors='coerce')
reviews['service_score'] = reviews['service_score'].apply(pd.to_numeric, errors='coerce')
reviews['atmosphere_score'] = reviews['atmosphere_score'].apply(pd.to_numeric, errors='coerce')
reviews['avg_price_per_person'] = reviews['price_per_person'].str.extract(r'-(\d+)\s*€')
reviews['avg_price_per_person'] = pd.to_numeric(reviews['avg_price_per_person'], errors='coerce').astype('Int64')


reviews.drop(columns = ['text_backup', 'local_guide_info', 'rating', 'author', 'recommended', 'date_text'], inplace = True)
reviews.reset_index(inplace=True)
reviews.rename(columns={'index': 'review_id', 'price_per_person':'price_per_person_category'}, inplace=True)

### Check null values

Fill missing values in specific columns with defaults (e.g., 1 for local_guide_reviews, 1 for rating_score).

In [18]:
# Check for missing values in each column
missing_values = reviews.isnull().sum()
print("Missing values per column:")
print(missing_values)

# Optionally, you can also check the percentage of missing values
missing_percentage = (reviews.isnull().mean() * 100).round(2)
print("Percentage of missing values per column:")
print(missing_percentage)


Missing values per column:
review_id                     0
review                        0
local_guide_reviews           0
rating_score                  0
service                       0
meal_type                     0
price_per_person_category     0
food_score                    9
service_score                 9
atmosphere_score              9
recommendations_list          0
date                          0
avg_price_per_person         11
dtype: int64
Percentage of missing values per column:
review_id                     0.00
review                        0.00
local_guide_reviews           0.00
rating_score                  0.00
service                       0.00
meal_type                     0.00
price_per_person_category     0.00
food_score                   30.00
service_score                30.00
atmosphere_score             30.00
recommendations_list          0.00
date                          0.00
avg_price_per_person         36.67
dtype: float64


In [7]:
# Fill NA values
reviews['local_guide_reviews'] = reviews['local_guide_reviews'].fillna(1)
reviews['rating_score'] = reviews['rating_score'].fillna(1)

### Variables distribution

Generate a summary of the numeric variables in the dataset. This provides insights into the distribution of ratings, review counts, and prices.

In [29]:
# Summary of numeric columns
print("Summary of numeric variables:")
display(reviews.describe())

# Summary of categorical columns
print("Distribution of categorical variables:")
for col in reviews.select_dtypes(include=['object']).columns:
    if col in ("review", 'recommendations_list', 'date'):
        continue
    print(f"\n{col} distribution:")
    print(reviews[col].value_counts())


Summary of numeric variables:


,review_id,local_guide_reviews,rating_score,food_score,service_score,atmosphere_score,avg_price_per_person
count,30.000000,30.000000,30.000000,21.000000,21.000000,21.000000,19.0
mean,14.500000,78.333333,4.800000,4.904762,4.857143,4.761905,37.894737
std,8.803408,82.964194,0.484234,0.300793,0.358569,0.538958,12.283208
min,0.000000,3.000000,3.000000,4.000000,4.000000,3.000000,30.0
25%,7.250000,28.750000,5.000000,5.000000,5.000000,5.000000,30.0
50%,14.500000,43.500000,5.000000,5.000000,5.000000,5.000000,30.0
75%,21.750000,97.750000,5.000000,5.000000,5.000000,5.000000,40.0
max,29.000000,311.000000,5.000000,5.000000,5.000000,5.000000,80.0


Distribution of categorical variables:

service distribution:
service
             25
Comí allí     5
Name: count, dtype: int64

meal_type distribution:
meal_type
          24
Comida     4
Cena       2
Name: count, dtype: int64

price_per_person_category distribution:
price_per_person_category
           11
20-30 €    10
30-40 €     6
40-50 €     2
70-80 €     1
Name: count, dtype: int64


### Saving clean data to processed folder

In [9]:
csv_file_path = '../data/processed/'
reviews.to_csv(csv_file_path + name + '_reviews.csv', index=False)

In [22]:
display(reviews.sample(20))

,review_id,review,local_guide_reviews,rating_score,service,meal_type,price_per_person_category,food_score,service_score,atmosphere_score,recommendations_list,date,avg_price_per_person
21,21,Fuimos el fin de semana a Zaragoza y nos gusta...,34,5,,,,NaN,NaN,NaN,[],2023-01-01,<NA>
27,27,"Ambiente muy bueno, sin ruidos y con un person...",33,5,Comí allí,Comida,,5.0,5.0,5.0,[],2023-01-01,<NA>
24,24,Una maravilla de restaurante. Personal amabilí...,31,5,Comí allí,Comida,20-30 €,5.0,5.0,5.0,[],2023-01-01,30
0,0,Espectacular!!! Probablemente uno de los resta...,35,5,,,30-40 €,5.0,5.0,5.0,[],2024-07-01,40
8,8,"Fuimos de celebración,el sitio es pequeño, de ...",40,5,,,30-40 €,5.0,5.0,5.0,[],2024-03-01,40
23,23,Un sitio muy agradable en el que probar comida...,11,5,Comí allí,Cena,70-80 €,NaN,NaN,NaN,[],2023-01-01,80
2,2,Cenamos en familia y fue una experiencia gastr...,3,5,,,20-30 €,5.0,5.0,5.0,"[Brandada de Bacalao, Cremoso de Chocolate Bla...",2024-08-01,30
22,22,Uno de los mejores lugares de Zaragoza en rela...,285,4,,Comida,30-40 €,4.0,4.0,4.0,"[Burrata Con Tomate Con Salmorejo, Panceta Asa...",2023-01-01,40
1,1,"La comida es excepcional! Las migas, las croqu...",28,5,,,,5.0,4.0,4.0,[],2024-07-01,<NA>
17,17,Fuimos en grupo a comer aprovechando el puente...,32,5,Comí allí,Comida,20-30 €,5.0,5.0,5.0,"[Croquetas de Jamón Y Gallina, Burrata Con Tom...",2023-01-01,30
